# Prompting Capabilities 

When you first start using Mistral models, your first interaction will revolve around prompts. The art of crafting effective prompts is essential for generating desirable responses from Mistral models or other LLMs. This guide will walk you through example prompts showing four different prompting capabilities. 

- Classification 
- Summarization 
- Personalization
- Evaluation


In [ ]:
! pip install mistralai==0.0.12

In [1]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [2]:
api_key = "TYPE YOUR API KEY"

In [3]:
def run_mistral(user_message, model="mistral-medium"):
    client = MistralClient(api_key=api_key)
    messages = [
        ChatMessage(role="user", content=user_message)
    ]
    chat_response = client.chat(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

## Classification
Mistral models can easily categorize text into distinct classes. In this example prompt, we can define a list of predefined categories and ask Mistral models to classify user inquiry. 



In [4]:
def user_message(inquiry):
    user_message = (
        f"""
        You are a bank customer service bot. Your task is to assess customer intent 
        and categorize customer inquiry after <<<>>> into one of the following predefined categories:
        
        card arrival
        change pin
        exchange rate
        country support 
        cancel transfer
        charge dispute
        
        If the text doesn't fit into any of the above categories, classify it as:
        customer service
        
        You will only respond with the predefined category. Do not include the word "Category". Do not provide explanations or notes. 
        
        ####
        Here are some examples:
        
        Inquiry: How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card as expected. Could you please provide information about the tracking process for my card, or confirm if there are any indicators to identify if the card has been lost during delivery?
        Category: card arrival
        Inquiry: I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any associated fees for foreign transactions.
        Category: exchange rate 
        Inquiry: What countries are getting support? I will be traveling and living abroad for an extended period of time, specifically in France and Germany, and would appreciate any information regarding compatibility and functionality in these regions.
        Category: country support
        Inquiry: Can I get help starting my computer? I am having difficulty starting my computer, and would appreciate your expertise in helping me troubleshoot the issue. 
        Category: customer service
        ###
    
        <<<
        Inquiry: {inquiry}
        >>>
        """
    )
    return user_message

### Strategies we used: 

- **Few shot learning**: Few-shot learning or in-context learning is when we give a few examples in the prompts, and the LLM can generate corresponding output based on the example demonstrations. Few-shot learning can often improve model performance especially when the task is difficult or when we want the model to respond in a specific manner. 
- **Delimiter**: Delimiters like ### <<< >>> specify the boundary between different sections of the text. In our example, we used ### to indicate examples and <<<>>> to indicate customer inquiry. 
- **Role playing**: Providing LLM a role (e.g., "You are a bank customer service bot.") adds personal context to the model and often leads to better performance. 

In [5]:
print(run_mistral(user_message(
    "I am inquiring about the availability of your cards in the EU, as I am a resident of France and am interested in using your cards. "
)))

country support


In [6]:
print(run_mistral(user_message("What's the weather today?")))

customer service


## Summarization

Summarization is a common task for LLMs due to their natural language understanding and generation capabilities. Here is an example prompt we can use to generate interesting questions about an essay and summarize the essay.


In [7]:
import requests
response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt')
essay = response.text

In [8]:
message = f"""
You are a commentator. Your task is to write a report on an essay. 
When presented with the essay, come up with interesting questions to ask, and answer each question. 
Afterward, combine all the information and write a report in the markdown format. 

# Essay: 
{essay}

# Instructions: 
## Summarize:
In clear and concise language, summarize the key points and themes presented in the essay.

## Interesting Questions: 
Generate three distinct and thought-provoking questions that can be asked about the content of the essay. For each question:
- After "Q: ", describe the problem 
- After "A: ", provide a detailed explanation of the problem addressed in the question.
- Enclose the ultimate answer in <>.

## Write a report 
Using the essay summary and the answers to the interesting questions, create a comprehensive report in Markdown format. 
"""

In [9]:
print(run_mistral(message))

**Summary:**

The essay is a reflection on the author's experiences and the lessons learned from working on different projects throughout their life. The author discusses their time spent writing short stories, programming, studying philosophy, and working in AI. They also touch on their brief foray into the world of art school and their eventual return to programming and the creation of Y Combinator. Throughout the essay, the author emphasizes the importance of focusing on projects that are interesting and meaningful, rather than chasing prestige.

**Interesting Questions:**

Q: What role did Lisp play in the author's life and career?
A: Lisp was a significant part of the author's life and career. They first learned Lisp in college, and it expanded their concept of a program and what was possible with code. The author also wrote a book about Lisp hacking, and they used Lisp to build the code editor for users to define their own page styles in Viaweb. Lisp was also the language used to

## Strategies we used: 

- **Step-by-step instructions**: This strategy is inspired by the chain-of-thought prompting that enables LLMs to use a series of intermediate reasoning steps to tackle complex tasks. It's often easier to solve complex problems when we decompose them into simpler and small steps and it's easier for us to debug and inspect the model behavior.  In our example, we break down the task into three steps: summarize, generate interesting questions, and write a report. This helps the language to think in each step and generate a more comprehensive final report. 
- **Example generation**: We can ask LLMs to automatically guide the reasoning and understanding process by generating examples with the explanations and steps. In this example, we ask the LLM to generate three questions and provide detailed explanations for each question. 
- **Output formatting**: We can ask LLMs to output in a certain format by directly asking "write a report in the Markdown format".



## Personlization

LLMs excel at personalization tasks as they can deliver content that aligns closely with individual users. In this example, we create personalized email responses to address customer questions. 

In [10]:
email = """
Dear mortgage lender, 

What's your 30-year fixed-rate APR, how is it compared to the 15-year fixed rate?

Regards,
Anna
"""

In [11]:
message = f"""

You are a mortgage lender customer service bot, and your task is to create personalized email responses to address customer questions.
Answer the customer's inquiry using the provided facts below. Ensure that your response is clear, concise, and 
directly addresses the customer's question. Address the customer in a friendly and professional manner. Sign the email with 
"Lender Customer Support."   


      
# Facts
30-year fixed-rate: interest rate 6.403%, APR 6.484%
20-year fixed-rate: interest rate 6.329%, APR 6.429%
15-year fixed-rate: interest rate 5.705%, APR 5.848%
10-year fixed-rate: interest rate 5.500%, APR 5.720%
7-year ARM: interest rate 7.011%, APR 7.660%
5-year ARM: interest rate 6.880%, APR 7.754%
3-year ARM: interest rate 6.125%, APR 7.204%
30-year fixed-rate FHA: interest rate 5.527%, APR 6.316%
30-year fixed-rate VA: interest rate 5.684%, APR 6.062%

# Email
{email}
"""

In [12]:
print(run_mistral(message))

Dear Anna,

Thank you for your inquiry regarding our mortgage rates.

Our current 30-year fixed-rate interest rate is 6.403% with an APR of 6.484%. In comparison, our 15-year fixed-rate interest rate is 5.705% with an APR of 5.848%.

Please note that the APR, or Annual Percentage Rate, is a broader measure of the cost of a mortgage loan that includes not only the interest rate but also other costs associated with the loan, such as points, fees, and mortgage insurance.

I hope this information is helpful. If you have any further questions, please don't hesitate to contact us.

Best regards,
Lender Customer Support


### Strategies we used: 
- Providing facts: Incorporating facts into prompts can be useful for developing customer support bots. It’s important to use clear and concise language when presenting these facts. This can help the LLM to provide accurate and quick responses to customer queries.

## Evaluation

There are many ways to evaluate LLM outputs. Here are three approaches for your reference: include a confidence score, introduce an evaluation step, or employ another LLM for evaluation.



## Include a confidence score
We can include a confidence score along with the generated output in the prompt. 

In [13]:
def run_mistral(user_message, model="mistral-medium"):
    client = MistralClient(api_key=api_key)
    messages = [
        ChatMessage(role="user", content=user_message)
    ]
    chat_response = client.chat(
        model=model,
        messages=messages,
        temperature=1
    )
    return (chat_response.choices[0].message.content)

In [14]:
message = f"""
You are a summarization system that can provide summaries with associated confidence scores.
In clear and concise language, provide three short summaries of the following essay, along with their confidence scores. 
You will only respond with a JSON object with the key Summary and Confidence. Do not provide explanations. 

# Essay: 
{essay}

"""

In [15]:
print(run_mistral(message))

{
  "Summary1": "The author discusses their experiences working on writing, programming, and art projects before and during college. They share anecdotes about their early attempts at programming on an IBM 1401, their love for the language Lisp, and their disillusionment with the field of philosophy. The author also reflects on their decision to study artificial intelligence in college and their eventual realization that the field was not what they had expected it to be.",
  "Confidence1": 0.85,
  "Summary2": "The author talks about their journey from writing short stories to programming and eventually studying AI in college. They discuss their initial disappointment with AI and how they switched to studying Lisp instead. They also touch upon their experiences in art school and their eventual decision to become an artist.",
  "Confidence2": 0.8,
  "Summary3": "The author reflects on their experiences working on various projects, including writing, programming, and art. They discuss the

### Strategies we used: 
- JSON output: For facilitating downstream tasks, JSON format output is frequently preferred. We can specify in the prompt that “You will only respond with a JSON object with the key Summary and Confidence.” Specifying these keys within the JSON object is beneficial for clarity and consistency.
- Higher Temperature: In this example, we increase the temperature score to encourage the model to be more creative and output three generated summaries that are different from each other.  


## Introduce an evaluation step 
We can also add a second step in the prompt for evaluation. 


In [16]:
message = f"""
You are given an essay text and need to provide summaries and evaluate them.

# Essay: 
{essay}

Step 1: In this step, provide three short summaries of the given essay. Each summary should be clear, concise, and capture the key points of the speech. Aim for around 2-3 sentences for each summary.
Step 2: Evaluate the three summaries from Step 1 and rate which one you believe is the best. Explain your choice by pointing out specific reasons such as clarity, completeness, and relevance to the speech content.


"""
print(run_mistral(message))

Summary 1:
The author shared their experiences in programming, writing, and art school. They started programming on an IBM 1401 in 9th grade and began writing short stories. They later switched to AI in college, but became disillusioned with it and turned to Lisp programming. After studying at an art school in Florence, they attended RISD where they learned about color and painting. However, they were disappointed with the painting department and eventually left to start their own software company, Viaweb, which was later acquired by Yahoo.

Summary 2:
The author describes their experiences in programming, AI, and art school. They started programming on an IBM 1401 in 9th grade and wrote short stories. After discovering Lisp programming in college, they became interested in AI but eventually became disillusioned with it. They then attended an art school in Florence and later RISD, where they were disappointed with the painting department. The author eventually left to start their softw

## Employ another LLM for evaluation
In production systems, it is common to employ another LLM for evaluation so that the evaluation step can be separate from the generation step.  

- Step 1: use the first LLM to generate three summaries 



In [17]:
message = f"""
Provide three short summaries of the given essay. Each summary should be clear, concise, and capture the key points of the essay.
Aim for around 2-3 sentences for each summary.

# essay: 
{essay}

"""
summaries = run_mistral(message)

In [18]:
print(summaries)

Summary 1:
The essay describes the author's experiences before college, focusing on their work with writing and programming. They wrote short stories as a beginner writer and experimented with programming on an IBM 1401 used for data processing. The author struggled to find a purpose for the 1401 and eventually shifted their focus to microcomputers, which allowed for direct user input and a more interactive programming experience.

Summary 2:
The author reflects on their early programming experiences, starting with writing simple programs on punch cards for an IBM 1401 and progressing to microcomputers. They learned to program in Fortran and later Lisp, eventually settling on the latter as their preferred language. The author's interest in artificial intelligence led them to write a book about Lisp, called "On Lisp". They also experimented with art classes at Harvard and discovered a passion for painting.

Summary 3:
The essay covers the author's background in writing and programming b

- Step 2: use another LLM to rate the generated summaries


In [19]:
message = f"""
You are given an essay and three summaries of the essay. Evaluate the three summaries and rate which one you believe is the best. 
Explain your choice by pointing out specific reasons such as clarity, completeness, and relevance to the essay content.

# Essay: 
{essay}

# Summries 
{summaries}

"""
print(run_mistral(message))

The best summary is Summary 3.

It provides a clear and concise overview of the essay, highlighting the author's experiences with writing and programming before college. It mentions the author's failed attempts at short story writing, their discovery of programming on an IBM 1401, and their shift to microcomputers. It also notes the author's interest in Lisp and their decision to write a book about it while attending Harvard for a Ph.D. in computer science. Finally, it acknowledges the author's unexpected passion for painting, discovered through art classes at Harvard.

Summary 1 is also fairly good, but it omits some important details, such as the author's interest in Lisp and their book about it. It also does not mention the author's passion for painting.

Summary 2 is the least accurate, as it omits many of the key details from the essay. It mentions the author's experiences with Fortran and Lisp, but does not provide any context or explanation for how they discovered these language

### Strategies we used: 
- **LLM chaining**: In this example, we chain two LLMs in a sequence, where the output from the first LLM serves as the input for the second LLM. The method of chaining LLMs can be adapted to suit your specific use cases. For instance, you might choose to employ three LLMs in a chain, where the output of two LLMs is funneled into the third LLM. While LLM chaining offers flexibility, it's important to consider that it may result in additional API calls and potentially increased costs.


